In [1]:
from openai import OpenAI
from prompts import non_native_style_aug_sys_prompt
from tqdm import tqdm

import os
import json
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

In [1]:
def non_native_augment(queries: dict[str, list[str]]):
    """Augment queries to mimic the language style of non-native English speakers."""
    res = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": non_native_style_aug_sys_prompt},
            {"role": "user", "content": json.dumps(queries, indent=4)},
        ],
        response_format={"type": "json_object"},
    )
    return res.choices[0].message.content

In [4]:
def persist(data: dict, file):
    """Persist data to the given file."""
    json.dump(data, file, ensure_ascii=False)
    file.write('\n')

In [5]:
def main(query_file_name: str):
    query_file = open(query_file_name, 'r')
    persist_file = open("non_native.jsonl", 'a')
    for line in tqdm(query_file):
        temp = json.loads(line)
        queries = {"customer_queries": temp["customer_queries"]}
        non_native = non_native_augment(queries)
        
        non_native = json.loads(non_native)
        non_native.update({"title": temp["title"], "tone": temp["tone"], "style": "non_native"})

        persist(non_native, persist_file)

In [6]:
main("queries.jsonl")
main("product_aug.jsonl")
main("tone_augmented.jsonl")

53it [03:38,  4.12s/it]
13it [02:49, 13.02s/it]
132it [17:52,  8.13s/it]


In [3]:
n = 0
with open("../queries_data/non_native.jsonl", 'r') as f:
    for json_str in f:
        obj = json.loads(json_str)
        n += len(obj['customer_queries'])

In [4]:
print(n)

3899
